In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content
!gdown --id 1t4hgtu292yglk2Ew70_L_-bNugzuBqHw -O data.zip

/content
Downloading...
From: https://drive.google.com/uc?id=1t4hgtu292yglk2Ew70_L_-bNugzuBqHw
To: /content/data.zip
886MB [00:06, 141MB/s]


In [ ]:
!nvidia-smi

Sun Jun 20 16:00:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd /content/drive/MyDrive/train_yolo_new2/darknet

/content/drive/MyDrive/train_yolo_new2/darknet


In [ ]:
!unzip data.zip

In [ ]:
%cd /content/drive/MyDrive/train_yolo_new2
!git clone https://github.com/AlexeyAB/darknet --depth=1
%cd darknet

/content/drive/MyDrive/train_yolo_new2
Cloning into 'darknet'...
remote: Enumerating objects: 1188, done.
remote: Counting objects: 100% (1188/1188), done.
remote: Compressing objects: 100% (958/958), done.
remote: Total 1188 (delta 271), reused 1050 (delta 223), pack-reused 0
Receiving objects: 100% (1188/1188), 5.33 MiB | 6.88 MiB/s, done.
Resolving deltas: 100% (271/271), done.
Checking out files: 100% (2032/2032), done.
/content/drive/MyDrive/train_yolo_new2/darknet


In [ ]:
%cd /content/drive/MyDrive/train_yolo_new2/darknet

/content/drive/MyDrive/train_yolo_new2/darknet


In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile
!sed -i "s/\$(CPP)/\$(CPP) -w/" Makefile
!sed -i "s/\$(CC)/\$(CC) -w/" Makefile
!sed -i "s/\$(NVCC)/\$(NVCC) -w/" Makefile
!make --silent

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a 

In [ ]:
!cp cfg/yolov4.cfg cfg/yolov4_training.cfg

In [ ]:
!sed -i -zE \
-e 's/(\nbatch)[^\n]+\n/\1=64\n/' \
-e 's/(\nsubdivisions)[^\n]+\n/\1=16\n/' \
-e 's/(\nmax_batches)[^\n]+\n/\1=25000\n/' \
-e 's/(\nsteps)[^\n]+\n/\1=20000,22500\n/' \
-e 's/(\nwidth)[^\n]+\n/\1=416\n/' \
-e 's/(\nheight)[^\n]+\n/\1=416\n/' \
-e 's/(\nfilters)[^\n]+(\n[^\n]+\n\n\n[^\n]+\n[^\n]+\n[^\n]+)(\nclasses)[^\n]+/\1=18\2\3=1/g' \
cfg/yolov4_training.cfg

In [ ]:
!printf "Person" > data/obj.names
!printf "classes=1\ntrain=/content/train.txt\nvalid=/content/valid.txt\nnames=data/obj.names\nbackup=backup/" > data/obj.data

In [ ]:
import os
from PIL import Image

DATASET_PATH = '/content/drive/MyDrive/mAP/mAP/input/ground-truth'
# Đọc tất cả các tệp tin trong thư mục dữ liệu train
images_list = os.listdir(DATASET_PATH)
# Lọc bỏ những tệp tin có đuôi .txt (chỉ quan tâm file hình)
images_list = [filename for filename in images_list if not filename[-4:] == '.txt']
# Thêm đường dẫn  vào trước tên tệp tin
images_list = [os.path.join(DATASET_PATH, filename) for filename in images_list]

for filename in images_list:
    
    # Đọc ảnh để lấy kích thước của ảnh
    img = Image.open(filename)
    width, height = img.size

    # Tìm tệp tin chứa nhãn dữ liệu tương ứng với ảnh
    annotation_file = os.path.splitext(filename)[0] + '.txt'

    # Đọc tệp tin chứa nhãn dữ liệu và tách thành từng dòng
    with open(annotation_file, 'r') as fi:
        lines = fi.read().splitlines()

    # Mở lại tệp tin nhãn để ghi đè
    with open(annotation_file, 'w') as fo:
        # Duyệt qua từng dòng trong tệp tin
        for line in lines:
            # Tách nội dung của từng dòng theo định dạng của BTC
            line = line.strip().split()
            # Chuyển đổi nội dung từ string sang int
            line = [int(x) for x in line]

            # Chuyển tọa độ xmin, ymin, xmax, ymax về tọa độ x_center, y_center, w, h
            # Lưu ý: YOLOv4 yêu cầu chuẩn hóa các tọa độ về khoảng [0; 1].
            line = [
                line[0],
                (line[3] + line[1])/2/width,    # x_center = (xmax + xmin)/2
                (line[4] + line[2])/2/height,   # y_center = (ymax + ymin)/2
                (line[3] - line[1])/width,      # w = (xmax - xmin)
                (line[4] - line[2])/height,     # h = (ymax - ymin)
            ]


            # Chuyển các tọa độ về kiểu string
            line = [str(x) for x in line]
            # Đưa về định dạng của YOLOv3
            line = ' '.join(line) + '\n'
            # Ghi vào tệp tin output
            fo.write(line)

In [ ]:
import os
import random
random.seed(42)

DATASET_PATH = '/content/drive/MyDrive/datatrain_yolo'
# Đọc tất cả các tệp tin trong thư mục dữ liệu train
images_list = os.listdir(DATASET_PATH)
# Lọc bỏ những tệp tin có đuôi .txt (chỉ quan tâm những file ảnh)
images_list = [filename for filename in images_list if not filename[-4:] == '.txt']
print(len(images_list))
# Thêm đường dẫn vào trước tên tệp tin
images_list = [os.path.join(DATASET_PATH, filename) for filename in images_list]

# Chọn 80% ngẫu nhiên ra làm tập dữ liệu train
train_list = random.sample(images_list, len(images_list)*8//10)
# Phần còn lại chính là tập dữ liệu validation
valid_list = list(set(images_list) - set(train_list))
print(len(train_list))
print(len(valid_list))

# Sắp xếp lại hai tập dữ liệu cho đẹp :)
train_list.sort()
valid_list.sort()
%cd /content/drive/MyDrive/train_yolo_new
# Cuối cùng chỉ cần xuất 2 tập dữ liệu ra 2 danh sách tương ứng
f = open("train.txt", "w")
f.write("\n".join(train_list))
f.close()

f = open("valid.txt", "w")
f.write("\n".join(valid_list))
f.close()

2188
1750
438
[Errno 2] No such file or directory: '/content/drive/MyDrive/train_yolo_new'
/content


In [ ]:
import os
import random
random.seed(42)

DATASET_PATH = '/content/data/images/val'
# Đọc tất cả các tệp tin trong thư mục dữ liệu train
images_list = os.listdir(DATASET_PATH)
# Lọc bỏ những tệp tin có đuôi .txt (chỉ quan tâm những file ảnh)
images_list = [filename for filename in images_list if not filename[-4:] == '.txt']
# Lọc bỏ những tệp tin có đuôi .txt (chỉ quan tâm những file ảnh)
images_list = [filename for filename in images_list]
# Thêm đường dẫn  vào trước tên tệp tin
images_list = [os.path.join(DATASET_PATH, filename) for filename in images_list]

# Chọn 80% ngẫu nhiên ra làm tập dữ liệu train
val_list = random.sample(images_list, len(images_list))


# Sắp xếp lại hai tập dữ liệu cho đẹp :)
val_list.sort()

%cd /content/drive/MyDrive/train_yolo_new2
# Cuối cùng chỉ cần xuất 2 tập dữ liệu ra 2 danh sách tương ứng
f = open("valid.txt", "w")
f.write("\n".join(val_list))
f.close()



/content/drive/MyDrive/train_yolo_new2


In [ ]:
%cd /content/drive/MyDrive/darknet
!wget https://pjreddie.com/media/files/darknet53.conv.74

/content/drive/MyDrive/darknet
--2021-06-19 12:33:32--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  57.5MB/s    in 2.7s    

2021-06-19 12:33:35 (57.5 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
import glob, os, shutil

source_dir = '/content/drive/MyDrive/data/labels/val' #Path where your files are at the moment
dst = '/content/drive/MyDrive/data/images/val' #Path you want to move your files to
files = glob.iglob(os.path.join(source_dir, "*.txt"))
for file in files:
    if os.path.isfile(file):
        shutil.copy(file, dst)

In [ ]:
%cd /content/drive/MyDrive/train_yolo_new2/darknet

/content/drive/MyDrive/train_yolo_new2/darknet


In [ ]:
import os
import random
random.seed(42)

WARMUP_PATH = '/content/drive/MyDrive/mAP/mAP/input/ground-truth'
# Đọc tất cả các tệp tin trong thư mục dữ liệu warmup
images_list = os.listdir(WARMUP_PATH)
# Lọc bỏ những tệp tin có đuôi .txt (chỉ quan tâm những file ảnh)
images_list = [filename for filename in images_list if not filename[-4:] == '.txt']
# Thêm đường dẫn /content/warmup/ vào trước tên tệp tin
images_list = [os.path.join(WARMUP_PATH, filename) for filename in images_list]

# Sắp xếp lại danh sách
images_list.sort()

# Cuối cùng là xuất danh sách này ra tệp tin txt
%cd /content/drive/MyDrive/train_yolo
f = open("test.txt", "w")
f.write("\n".join(images_list))
f.close()

/content/drive/MyDrive/train_yolo


In [ ]:
%cd /content/drive/MyDrive/train_yolo_new2
!wget https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip

/content/drive/MyDrive/train_yolo_new2
--2021-06-27 16:22:24--  https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip
Resolving www.cis.upenn.edu (www.cis.upenn.edu)... 158.130.69.163, 2607:f470:8:64:5ea5::d
Connecting to www.cis.upenn.edu (www.cis.upenn.edu)|158.130.69.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53723336 (51M) [application/zip]
Saving to: ‘PennFudanPed.zip’

PennFudanPed.zip    100%[===================>]  51.23M  21.7MB/s    in 2.4s    

2021-06-27 16:22:27 (21.7 MB/s) - ‘PennFudanPed.zip’ saved [53723336/53723336]



In [ ]:
%cd /content/drive/MyDrive/train_yolo_new2/darknet

/content/drive/MyDrive/train_yolo_new2/darknet


In [ ]:
!./darknet detector train data/obj.data cfg/yolov4_training.cfg backup/yolov4_training_best.weights -thresh 0.25 -dont_show -save_labels < /content/drive/MyDrive/train_yolo_new2/test.txt> result.txt

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4_training.cfg backup/yolov4_training_best.weights -thresh 0.25

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x